In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

ID              url                                              title  \
0   0          m.kp.md  Экс-министр экономики Молдовы - главе МИДЭИ, ц...   
1   1        www.kp.by  Эта песня стала известна многим телезрителям б...   
2   2    fanserials.tv  Банши 4 сезон 2 серия Бремя красоты смотреть о...   
3   3  colorbox.spb.ru                              Не Беси Меня Картинки   
4   4    tula-sport.ru  В Новомосковске сыграют следж-хоккеисты алекси...   

   label  
0      0  
1      0  
2      0  
3      0  
4      0

In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

ID                url  \
0  135309  www.kommersant.ru   
1  135310    urexpert.online   
2  135311      imperimeha.ru   
3  135312  national-porn.com   
4  135313            2gis.ru   

                                               title  
0  Шестой кассационный суд в Самаре начнет работу...  
1  Что такое индексация алиментов, кем и в каких ...  
2                  Женщинам | Империя Меха - Part 12  
3  Небритые, волосатые киски: Порно всех стран и ...  
4                                                 67

In [4]:
from sklearn.metrics import f1_score

In [5]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["label"].values

# Simple baseline

In [6]:
y_pred = [int("порно" in text) for text in X_train]

In [7]:
f1_score(y_train, y_pred)

0.31242758551206484

### Submit

In [8]:
test_df["label"] = ["порно" in text for text in X_test]

test_df[["ID", "label"]].to_csv("simple_baseline.csv", index=False)

!cat simple_baseline.csv | head

ID,label
135309,False
135310,False
135311,False
135312,False
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe


### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

# ML baseline

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()

model = MultinomialNB()

In [10]:
X_train

array(['Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�',
       'Эта песня стала известна многим телезрителям благодаря сериалу Диверсант-2',
       'Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!', ...,
       '1941 Plymouth Special Deluxe Hot Rod, Automatic,Small Block,Power Steering, A/C for sale: photos, technical specifications, description',
       "Купить It's Skin Сыворотка питательная Power 10 Formula YE Effector, 30 мл в Москве - WOWCREAM",
       'Технический спирт в канистрах и флаконах, купить в Москве'],
      dtype=object)

In [11]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [12]:
X_train_vectorized

<135309x189852 sparse matrix of type '<class 'numpy.int64'>'
	with 1202659 stored elements in Compressed Sparse Row format>

In [13]:
feature_names = np.array(vectorizer.get_feature_names_out())

In [14]:
feature_names

array(['00', '000', '0000', ..., '𝔸𝕟𝕘𝕖𝕝𝕒', '𝔾𝕌𝕊𝕋𝔸𝕍', '𝕆𝕃𝔾𝔸'], dtype=object)

In [15]:
id_ = 42

print(X_train[id_])

x_vector = X_train_vectorized.getrow(id_).toarray()[0]

[feature for feature in feature_names[x_vector > 0]]

Презентация на тему "Приближенное значение. Абсолютная и относительная погрешнос


['абсолютная',
 'значение',
 'на',
 'относительная',
 'погрешнос',
 'презентация',
 'приближенное',
 'тему']

In [16]:
%%time

model.fit(
    X_train_vectorized,
    y_train
)

y_pred = model.predict(
    X_train_vectorized
)

CPU times: user 52.7 ms, sys: 8.11 ms, total: 60.8 ms
Wall time: 58.9 ms


In [17]:
f1_score(y_train, y_pred)

0.9036641864382985

### Submit

In [18]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["label"] = model.predict(X_test_vectorized)

test_df[["ID", "label"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

ID,label
135309,0
135310,0
135311,0
135312,1
135313,0
135314,0
135315,0
135316,0
135317,0
cat: stdout: Broken pipe
